## Step 0: Install docker and place sudo password in ~/.sudopassword

### Removed previous run containers

In [27]:
!sudo -S docker rm /zookeeper-server < ~/.sudopassword
!sudo -S docker rm /kafka-server < ~/.sudopassword
!sudo -S docker rm /event-stream < ~/.sudopassword
!sudo -S docker network rm app-tier < ~/.sudopassword

[sudo] password for kalinga: Error: No such container: zookeeper-server
[sudo] password for kalinga: Error: No such container: kafka-server
[sudo] password for kalinga: Error: No such container: event-stream
[sudo] password for kalinga: Error: No such network: app-tier


## Step 1: Create a network

In [28]:
!sudo -S docker network create app-tier --driver bridge < ~/.sudopassword

[sudo] password for kalinga: 8ce090061d34f7f570f31ea4fd963fedc611395db9ebb865ca679432a81567b7


## Step 2: Launch the Zookeeper server instance

In [29]:
!sudo -S  docker run -d --name zookeeper-server --network app-tier -e ALLOW_ANONYMOUS_LOGIN=yes bitnami/zookeeper:latest < ~/.sudopassword

[sudo] password for kalinga: d64933adb807683cb02ab9d6857e343f57ffa42fe8a72e2f8082b31eecd9914c


## Step 3: Launch the Kafka server instance

In [30]:
!sudo -S docker run -d --name kafka-server \
    --network app-tier \
    -e ALLOW_PLAINTEXT_LISTENER=yes \
    -e KAFKA_CFG_ZOOKEEPER_CONNECT=zookeeper-server:2181 \
    bitnami/kafka:latest < ~/.sudopassword

[sudo] password for kalinga: 055ddb350b3ec8292e4372b9f052fb299049835890aa26261bc154e8046eba94


## Step 4: Launch event-stream  instance

In [31]:
!sudo -S docker build -t kalinga/event-stream:v0.1 . < ~/.sudopassword

[sudo] password for kalinga: Sending build context to Docker daemon  39.42kB
Step 1/8 : FROM python:3
 ---> b162b5322f1c
Step 2/8 : LABEL Author="Kalinga Bhusan Ray"
 ---> Using cache
 ---> 55e95b0d90db
Step 3/8 : LABEL E-Mail="kalinga-bhusan.ray@tu-ilmenau.de"
 ---> Using cache
 ---> 5e490150ac8b
Step 4/8 : LABEL Desc="This is an image for event stream engiene"
 ---> Using cache
 ---> 430a9dc7ecdd
Step 5/8 : RUN apt-get update &&     apt-get install -y vim
 ---> Using cache
 ---> 63f97c80a8c5
Step 6/8 : ADD process_event_stream.py /
 ---> Using cache
 ---> e1d9d77b1ff4
Step 7/8 : RUN pip install faust typing_extensions
 ---> Using cache
 ---> 4ffa844c6546
Step 8/8 : EXPOSE 80
 ---> Using cache
 ---> f1ad588d40d8
Successfully built f1ad588d40d8
Successfully tagged kalinga/event-stream:v0.1


In [38]:
!sudo -S docker run -it -d --name event-stream \
    --network app-tier \
    kalinga/event-stream:v0.1 /bin/bash < ~/.sudopassword

[sudo] password for kalinga: c9967c731f8ff46d4a85fbb985b66b58028b6cf8f8a2b32e8f03683fee042089


In [39]:
!sudo -S docker ps  < ~/.sudopassword

[sudo] password for kalinga: CONTAINER ID        IMAGE                       COMMAND                  CREATED             STATUS              PORTS                                    NAMES
c9967c731f8f        kalinga/event-stream:v0.1   "/bin/bash"              17 seconds ago      Up 3 seconds        80/tcp                                   event-stream
055ddb350b3e        bitnami/kafka:latest        "/entrypoint.sh /run…"   11 minutes ago      Up 11 minutes       9092/tcp                                 kafka-server
d64933adb807        bitnami/zookeeper:latest    "/entrypoint.sh /run…"   11 minutes ago      Up 11 minutes       2181/tcp, 2888/tcp, 3888/tcp, 8080/tcp   zookeeper-server


In [33]:
!ls

Dockerfile	    operations		     README.md
event-stream.ipynb  process_event_stream.py  test.py


#### [In terminal]
docker exec -it event-stream bash   
docker exec -it kafka-server bash 


In [ ]:
kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic transaction --producer.config /opt/bitnami/kafka/conf/producer.properties


In [ ]:
!sudo -S docker cp event-stream:/process_event_stream.py . < ~/.sudopassword

In [7]:
!sudo -S docker start /zookeeper-server < ~/.sudopassword
!sudo -S docker start /kafka-server < ~/.sudopassword
!sudo -S docker start /event-stream < ~/.sudopassword

[sudo] password for kalinga: /zookeeper-server
[sudo] password for kalinga: /kafka-server
[sudo] password for kalinga: /event-stream


In [9]:
#!sudo -S docker container ls --all < ~/.sudopassword

In [11]:
!sudo -S docker ps < ~/.sudopassword

[sudo] password for kalinga: CONTAINER ID        IMAGE                       COMMAND                  CREATED             STATUS              PORTS                                    NAMES
c9967c731f8f        kalinga/event-stream:v0.1   "/bin/bash"              4 hours ago         Up 2 minutes        80/tcp                                   event-stream
055ddb350b3e        bitnami/kafka:latest        "/entrypoint.sh /run…"   4 hours ago         Up 2 minutes        9092/tcp                                 kafka-server
d64933adb807        bitnami/zookeeper:latest    "/entrypoint.sh /run…"   4 hours ago         Up 2 minutes        2181/tcp, 2888/tcp, 3888/tcp, 8080/tcp   zookeeper-server
